# Graph Neural Networks


---
**Learning Objectives**
* You understand how molecules can be represented as graphs. 
* You will understand how basic Graph Neural Networks work.
* You will be able to write a Graph Neural Network as a Pytorch class.
---


Graph Neural Networks are still a relatively new class of algorithms. Intuitively, molecules can be represented very easily as a (mathematical) graph. The bonds of a molecule correspond to the edges of the graph and the atoms to the nodes. <br> For computers, however, graphs are not as easy to read as, for example, a Smiles `string`. For graph neural networks we need at least two matrices to adequately represent moleculs. One is the adjacency matrix, which represents the connections between the atoms (i.e. the bonds). The other matrix is the feature matrix. This is where information about individual atoms can be stored.

<center>
<img src="https://www.researchgate.net/profile/Jorge_Galvez2/publication/236018587/figure/fig1/AS:299800013623305@1448489301609/The-chemical-graph-and-adjacency-matrix-of-the-isopentane.png" style="width: 600px;">
</center>
<h8><center>Galvez et. al. 2010</center></h8><br><br>

For today's example we use again the data from the Tox21 Challenge. 

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils import data
import math
from sklearn.metrics import roc_auc_score
import sys
from os.path import exists
if 'google.colab' in sys.modules:
    !pip install rdkit==2022.3.4
    if exists("utils.py") == False:
        !wget https://raw.githubusercontent.com/kochgroup/intro_pharma_ai/main/utils/utils.py
    %run ../utils/onehotencoder.py
else:
    %run ../utils/onehotencoder.py
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

np.set_printoptions(linewidth=300)

In [ ]:
# Laden der Daten
data_tox = pd.read_csv("https://raw.githubusercontent.com/filipsPL/tox21_dataset/master/compounds/sr-mmp.tab", sep = "\t")
data_tox = data_tox.iloc[:,1:] # all columns except the first one (index 0) are selected
data_tox.columns = ["smiles", "activity"]
data_tox.head()

## Adjacency Matrix and One-Hot Encoded Feature Matrix

Unfortunately, we can't really use moleucles represented as Smiles this time. We need convert molecules into a graph representation. RDKit provides some functionalities that make it easier to work with graphs. For example, there is a function that creates adjacency matrices for molecules. We have already imported this one above.

In [ ]:
mols = [Chem.MolFromSmiles(x) for x in data_tox['smiles']]
A = [GetAdjacencyMatrix(x) for x in mols]
print(A[1])

As features for the atoms we use only the atomtype. We will also one-hot encode these.

For the one-hot encoding of the atoms we use the already written function `onehotencode()`.
Similar to the encoding of Smiles tokens in RNNs, only the atomtype of each atom is recorded here. These atoms are then represented as one-hot encoded vectors.
Per molecule, these vectors are combined to form a matrix.

In [ ]:
feat = onehotencode(mols)
feat[1]

Oben können Sie sehen, wie eine Featurematrix für ein Molekül aussieht.
Wenn wir uns die `.shape` ansehen, können wir sehen, dass dieses Molekül aus `29` Atomen besteht. Die Anzahl der Spalten lässt uns wissen, dass es insgesamt 25 Atomtypen im Datensetz gibt. Die Anzahl der Spalten (Anzahl der Features) muss für alle Moleküle gleich sein, sonst können wir die Moleküle nicht durch das Netzwerk führen.

In [ ]:
feat[1].shape

You may have noticed that there are still zeros on the diagonal of the adjacency matrix. In a graph convolution, however, not only the features of the neighboring atoms but also those of the central atom are to be included in the aggregation. For this, ones are needed on the diagonal of the adjacency matrix. 
With the function `np.fill_diagonal(matrix, value)` you can change the values of the diagonals of a matrix. 

In [ ]:
for matrix in A:
    np.fill_diagonal(matrix, 1)
print(A[1])

## Graph Convolution

Now we want to pass the information of each node to the neighboring nodes along the edges. This is called a graph convolution. One way to this, is using the following operation for the forward pass: $$\hat{X} = \hat{D}^{-1}\hat{A}XW$$.

Here $\hat{A}$ is our adjacency matrix, with ones on the diagonal. $X$ is the feature matrix and $W$ are the weights that Pytorch initializes. What we are still missing is $D$, the $D$egreen matrix. This matrix contains the number of bonds each atom has in the molecule. These values are placed on the diagonal of the degreematrix.
The number of bonds of each molecule can be easily calculated from the adjacency matrix. To do this, one must sum over the rows or columns of the matrix $\hat{A}$. This sum gives us the degree of an atom. To be exact, it is the degree plus one, since you have already filled the diagonal of the adjacency matrix with ones.
To create the degree matrix, you must calculate the sums of the columns in each adjacency matrix and place them on the diagonal of a new matrix.


This process is relatively easy to do with `numpy`. `np.sum(A[i], axis=0)` calculates the sum per column and `np.diag()` creates a matrix from a 1D array with the values of the 1D array on the diagonal. You can use the two functions in combination to create the degreematrixes.

In [ ]:
D =[]
for matrix in A:
    D.append(np.diag(np.sum(matrix, axis=1)))
print(D[1])


But we do not need $\hat{D}$. We need the inverse of this matrix. Without $\hat{D}^{-1}$, $\hat{A}X$ would sum the features across all connected nodes. This would result in atoms with more neighbors having larger feature values. By including $\hat{D}^{-1}$, the aggregated features are averaged by the number of neighboring atoms. `np.linalg.inv()` is needed to get the inverse of the matrix `D`.

Before we start building a network, we need to perform one more step. To save computational effort, we can precompute $\hat{D}^{-1}\hat{A}$ before training the netowrk. This can save time since this step will not be repeatedly computed during training.





In [ ]:
DA = []
for i in range(len(D)):
    DA.append(np.matmul(np.linalg.inv(D[i]),A[i]))
DA[0]

We now have the list of adjacency matrices `DA`, contains the information about the structure of the molecule. The list `feat` contains the features, i.e. the information about the individual atoms. And finally, we create a list `labels`. This list contains the information that we want to predict (toxic vs. non-toxic).

In [ ]:
DA = [torch.tensor(x,dtype=torch.float32) for x in DA] 
feat = [torch.tensor(x,dtype=torch.float32) for x in feat] 
labels = [torch.tensor([x], dtype=torch.float32) for x in data_tox['activity']]

## Graph Convolution Layer

We want to take advantage of PyTorch for graph convolution. Like last week, we will use classes to do this.
We used a class to create a network  made out of different layers (e.g. `nn.Linear()` or `nn.GRU`). 
These layers are already given in PyTorch, but we can also create our own layers. For this we need to write a new PyTorch class.

Our goal is to program something like `nn.Linear()` so that we can add graph convolution to our repertoire of layers (Linear, RNN, GRU, Dropout, ...).

Again we can use the class `nn.Module` as a basis for our Graph Convolution. 
We start with the minimal framework:



```python
class GraphConvolution(nn.Module):
    pass
```


In order to properly access the functions of the parent `nn.Module` class, we need to initialize it with `super().__init__()`. Of course, our convolutional layer needs to know how big the input is, and how big the output should be. 


```python
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_featuers = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
```

Now we can write the `forward()` function. Here the actual convolution happens, where the nodes with the features `x` are aggregated over their neighboring nodes using a matrix multiplication with the adjacency matrix `adj`. By including the learnable `weights` we can optimize the convolution during training.

```python
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
```
With this, our `GraphConvolution` class would already work. But since we do not define a network now, but a new layer, we must additionally specify how the weights and biases are to be initialized.
We initialize the weights with the function `reset_parameters()`. The new function `__repr__()` provides a  graphically overview what our layer looks like after it has been initialized. 


Our final class will look like this:

In [ ]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()
    
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(0, stdv)
        self.bias.data.uniform_(-stdv, stdv)
        
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + 'in_features=' + str(self.in_features) + ', ' \
               + 'out_features=' + str(self.out_features) + ')'

We can also check if our Graph Convolution Layer is already working.
First, we save a signle feature matrix and an adjacency matrix as an example.

In [ ]:
feat_example = feat[1]
adj_example = DA[1]
print('Features:', feat_example.shape)
print('DA:', adj_example.shape)

We now initialize a `GraphConvolution`. Note that the input size, of the first layer is equal to the size of the feature vector (number of columns) (`25`). In that regard it behaves like a `nn.Linear` layer.

In [ ]:
conv = GraphConvolution(25, 100)
conv

We can now take the example through convolution. You will see that the dimension of the features will have increased to 100.

In [ ]:
output = conv(feat_example, adj_example)
print('\nOutput:', output.size())

## Graph Neural Network

To create a network now, we can use the class `nn.Module` again. 

Last week we put together our own autoencoder. This week we use this type of class to connect multiple graph convolutions together. However, it is important to note that, as with regular CNNs, we cannot use only convolution layers. In the CNN itself, we need to "flatten" the tensor at the end. This gives us a vector that is then passed through a linear layer. The same is true for graph convolutions. However, to pass the output of these graph convolutions into a linear layer, we cannot simply use PyTorch's `flatten`, but we will take the average across all columns.

This is done with the function `aggregate()`.

You can see in the network we use two graph convolution and then a linear layer.
In the forward pass, the input `(x, adj)` is first passed through the inital graph convolution, then a ReLU function is applied. The same is then repeated for the second convolution. 
Now we apply the function `aggregate`. This calculates the mean value of each feature. Thus the output of this layer has the size `[1,100]` in the example from above.
Finally, we use the linear layer to make the actual prediction.

In [ ]:
class GraphNN(nn.Module):
    def __init__(self):#in_features, out_features, size_labels):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): # we use mean aggregation, max or min could also be used as hyperparameter
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x

Now we will quickly divide the dataset into a training set and a test set. To do this, we simply use the first 1800 molecules as the training set and the rest as the test set. It is important that we do not use minibtaches in this example. The reason for this is that the feature matrices of each molecule are of different sizes (have different numbers of rows). This means that we cannot simply store them as a 3D tensor, as is the case with CNNs, for example. 

There are ways to solve this problem, but they are not relevant for this notebook. 

In [ ]:
train_feat = feat[:1800]
train_DA = DA[:1800]
train_labels = labels[:1800]


test_feat = feat[1800:]
test_DA = DA[1800:]
test_labels = labels[1800:]

In [ ]:
gnn = GraphNN()
loss_funktion= nn.BCEWithLogitsLoss()
optimizer=optim.Adam(gnn.parameters(), lr =0.01)

In [ ]:
EPOCHS = 20

for i in range(EPOCHS):
    loss_list_train = []
    acc_list_train= []
    gnn.train()
    for k in range(len(train_feat)):
        optimizer.zero_grad()
    
        output=gnn(train_feat[k], train_DA[k]).flatten()

        loss=loss_funktion(output,train_labels[k])
        loss.backward()
        loss_list_train.append(loss.item())
        optimizer.step()

        acc_list_train.append(np.sum((torch.round(torch.sigmoid(output)) == train_labels[k]).detach().numpy()))
    loss_list_test = []
    acc_list_test= []
    gnn.eval()
    for k in range(len(test_feat)):
    
        output=gnn(test_feat[k], test_DA[k]).flatten()

        loss=loss_funktion(output,test_labels[k])
        loss_list_test.append(loss.item())
 

        acc_list_test.append(np.sum((torch.round(torch.sigmoid(output)) == test_labels[k]).detach().numpy()))
            
        
    print(i,"Train Loss: %.2f Train Accuracy: %.2f Test Loss: %.2f Test Accuracy: %.2f"
        % (np.mean(loss_list_train), np.mean(acc_list_train),np.mean(loss_list_test), np.mean(acc_list_test)))

As you can see, the training takes a long time and is not very successful. The model presented here is a very, very simple model. In fact, more complex graph convolutions are usually used. Additionally, a variety of features, not only the atomtype,are used as input in the feature matrix. 

For more complex Graph Neural Netwroks libraries such as [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) or [Deep Graph Library](https://www.dgl.ai/) exist. Both are an extension to PyTorch. They contain important functionalities that are relevant for dealing with graphs. In addition, the libraries contain the most important graph layers. This way you do not have to program the layers yourself.

## Exercise:

Here you can see our Graph Neural Network.
The problem is that this network offers no flexibility.
The weight matrices of the network will always have the same dimensions.
Dropout is also missing. This time we don't need a batchnorm, tho, because we don't use minibatches.


```python
class GraphNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): 
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x
```
 
Can you rewrite the network so that we can flexibly adjust the input as well as the sizes of the graph convolutions?
You can test if your network still works with the `example_DA` and `example_feat`.

In [ ]:
example_DA = test_DA[0]
example_feat = test_feat[0]

In [ ]:
class GraphNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): 
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x